# Generate footprints from NARA csv of declass3 panoramic camera strips

Should be 4 csv files (KH-9 Hexagon Final 1.csv, etc), each containing ~400-700K records, mixed mapping and panoramic frames

Pan strips are split into 15 degree intervals in cross-track direction.  Merged these in dissolve output

Some polygons E and W lon values don't make sense.  Should do more careful check to make sure S lat is less than N lat, W lon is less than E lon, etc.

Also, the first image in a set of strips is often offset from the larger block.  Unclear if this is issue with csv coords or if this is due to how original images were acquired.

In [102]:
%matplotlib inline
import geopandas as gpd
from shapely import wkt
import pandas as pd
import dask.dataframe as dd
from pygeotools.lib import geolib
import numpy as np
import glob
import re
import os

In [103]:
csv_fn_list = glob.glob("KH-9*csv")
csv_fn_list

['KH-9_Hexagon_Final_3.csv',
 'KH-9_Hexagon_Final_2.csv',
 'KH-9_Hexagon_Final_1.csv',
 'KH-9_Hexagon_Final_4.csv']

In [104]:
csv_fn = csv_fn_list[2]
out_fn = os.path.splitext(csv_fn)[0]+'_clean.gpkg'
csv_fn

'KH-9_Hexagon_Final_1.csv'

In [105]:
#Read csv and clean up formatting (whitespace)
df = pd.read_csv(csv_fn, skipinitialspace=True)
df.columns = [x.replace(' ', '').upper() for x in df.columns]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.sort_values(by='DATE')

,DATE,MISSION,BUCKET,SYSTEM,OP,FRAME,SUBFRAME,CAMERA,SELAT,SELON,NELAT,NELON,NWLAT,NWLON,SWLAT,SWLON
0,19710616,1201,1,KH-9,1,1,0,A,244431N,1194218W,250234N,1194608W,245544N,1203231W,243932N,1203003W
801,19710616,1201,1,KH-9,5,10,0,A,415204N,1173524W,415642N,1181358W,414434N,1181500W,414116N,1173719W
800,19710616,1201,1,KH-9,5,9,-45,F,420654N,1185313W,421521N,1195414W,420007N,1200101W,415334N,1185804W
799,19710616,1201,1,KH-9,5,9,-30,F,420050N,1180928W,420654N,1185313W,415334N,1185804W,414854N,1181301W
798,19710616,1201,1,KH-9,5,9,-30,A,421225N,1185631W,421958N,1195931W,420401N,1195750W,415840N,1185624W
797,19710616,1201,1,KH-9,5,9,-15,F,415537N,1173155W,420050N,1180928W,414854N,1181301W,414452N,1173419W
796,19710616,1201,1,KH-9,5,9,-15,A,420701N,1181124W,421225N,1185631W,415840N,1185624W,415450N,1181225W
802,19710616,1201,1,KH-9,5,10,0,F,414124N,1165700W,414521N,1173440W,413433N,1173701W,412916N,1165848W
795,19710616,1201,1,KH-9,5,9,45,A,414937N,1150506W,415504N,1160827W,414152N,1161319W,413437N,1151218W
793,19710616,1201,1,KH-9,5,9,30,A,415504N,1160827W,415902N,1165345W,414706N,1165703W,414152N,1161319W


In [106]:
#Run tests on a subset
#df = df[0:10000]
df.iloc[0]

DATE        19710616
MISSION         1201
BUCKET             1
SYSTEM          KH-9
OP                 1
FRAME              1
SUBFRAME           0
CAMERA             A
SELAT        244431N
SELON       1194218W
NELAT        250234N
NELON       1194608W
NWLAT        245544N
NWLON       1203231W
SWLAT        243932N
SWLON       1203003W
Name: 0, dtype: object

In [107]:
#Attempt to link can numbers with each frame
#can_db_fn = '/Users/dshean/Documents/UW/CONUS/declassified_imagery/NARA_records_20181217/mdb_export/KH-9\ Missions.xlsx'
#can_df = pd.read_excel(can_db_fn)

In [108]:
print(df.columns)
print(df.shape)
df.nunique()

Index(['DATE', 'MISSION', 'BUCKET', 'SYSTEM', 'OP', 'FRAME', 'SUBFRAME',
       'CAMERA', 'SELAT', 'SELON', 'NELAT', 'NELON', 'NWLAT', 'NWLON', 'SWLAT',
       'SWLON'],
      dtype='object')
(665042, 16)


DATE           365
MISSION          7
BUCKET           4
SYSTEM           1
OP             731
FRAME          202
SUBFRAME         9
CAMERA           2
SELAT        78469
SELON       179084
NELAT        78668
NELON       178803
NWLAT        78559
NWLON       178813
SWLAT        78385
SWLON       179070
dtype: int64

In [109]:
df['CAMERA'] = df['CAMERA'].fillna('T')
df['CAMERA'].value_counts()

F    349021
A    316021
Name: CAMERA, dtype: int64

In [110]:
#Remove terrain camera footprints
df_T = df[df['CAMERA'] == 'T']
df = df[df['CAMERA'] != 'T']
#Process terrain camera footprints
#df = df_T

In [111]:
def str2ll(dms_str):
    #print(dms_str)
    dms_str = str(dms_str)
    #Set sign appropriately so we have (-180,180) lon and (-90,90) lat
    #Note that some of these may be labeled incorrectly near 0/180 lon and 0 lat
    sign = 1
    if re.search('[swSW]', dms_str):
        sign = -1
    #If length is 7 characters, assume we are dealing with latitude string
    if len(dms_str) == 7:
        (degree, minute, second) = dms_str[0:2], dms_str[2:4], dms_str[4:6]
    #If length is 8 characters, assume we are dealing with latitude string
    elif len(dms_str) == 8:
        (degree, minute, second) = dms_str[0:3], dms_str[3:5], dms_str[5:7]
    #Convert DMS to decimal degrees, simple formula, could do here for simplicity
    dd = sign*geolib.dms2dd(int(degree), int(minute), float(second))
    return dd

In [112]:
#Example coordinate conversion
dms_str = df['SWLON'].iloc[0]
print(dms_str)
str2ll(df['SWLON'].iloc[0])

IndexError: single positional indexer is out-of-bounds

In [ ]:
#Convert all coordinates to decimal degrees
df2 = df[['SELAT', 'SELON', 'NELAT', 'NELON', 'NWLAT', 'NWLON', 'SWLAT', 'SWLON']].applymap(str2ll)

In [ ]:
#Lots of issues with corner longitude values (e.g., west is greater than east)
#but doesn't seem to be an issue after converting to geom
#idx = (df2['SELON']+180. < df2['SWLON']+180.) | (df2['NELON']+180. < df2['NWLON']+180.)
#df2[idx]

In [ ]:
lon_idx = [col for col in df2.columns if 'LON' in col]
lon_idx.append(lon_idx[0])
lat_idx = [col for col in df2.columns if 'LAT' in col]
lat_idx.append(lat_idx[0])

In [ ]:
#Identify individual feature geom that cross the antimeridian
#idx = ((df2['SELON'] < 0.) | (df2['NELON'] < 0.)) & ((df2['SWLON'] > 0.) | (df2['NWLON'] > 0.))
antimeridian_idx = (df2[lon_idx].min(axis=1) - df2[lon_idx].max(axis=1)).abs() > 180
df2[antimeridian_idx].shape

In [ ]:
#Should use Fiona/geopandas functionality for these features spanning -180/180
#define as projected coords, then reproject to wgs84, which should split to multipolygon

In [ ]:
def fix_corners(ds):
    #Need to split into multipolygon
    #Determine 180lat at 180 from interpolation of NELAT and NWLAT
    #Fiona has this functionality in transform (antimeridian_cutting)
    return ds

In [ ]:
#Format list of corner coordinates to wkt polygon string
def dd2wkt(ds):
    #ds = fix_corners(ds)
    return 'POLYGON(({0}))'.format(', '.join(['{0} {1}'.format(*a) for a in zip(ds[lon_idx],ds[lat_idx])]))

In [ ]:
#This has long runtime, preparing strings for each row of dataframe
#df2_dd = dd.from_pandas(df2, npartitions=8)
geom_col = df2.apply(dd2wkt, axis=1)
#geom_col.compute()

In [ ]:
#Create geometries using shapely wkt.loads function
df['geom'] = geom_col.apply(wkt.loads)

In [ ]:
#Create geopandas dataframe with crs set
gdf = gpd.GeoDataFrame(df, geometry='geom', crs={'init' :'epsg:4326'})

In [ ]:
#i=403623
#print(gdf.loc[i].geom)
#gdf.loc[i]

In [ ]:
#Remove invalid geometries
#Should investigate and fix if possible
print("Input:", gdf.shape)
gdf = gdf[~antimeridian_idx]
print("Antimeridian:", gdf.shape)
valid_geom_idx = gdf.is_valid
gdf = gdf[valid_geom_idx]
print("Valid:", gdf.shape)

In [ ]:
gdf[::10].plot(figsize=(16,6))

In [ ]:
#Now dissolve 15-degree strips to create single footprint for each image
gdf2 = gdf.dissolve(by=['MISSION','DATE','OP','FRAME','CAMERA'], aggfunc={'SUBFRAME':(np.size, np.sum, np.ptp)}, as_index=False)
print(gdf2.shape)
gdf2
#Need to deal with Multipolygon vs polygon output here

In [ ]:
#Clean up aggregation column names
gdf2.columns = ['_'.join(i) if isinstance(i, tuple) else i for i in gdf2.columns]
gdf2.columns

In [ ]:
#The section angle starts at left edge
subframe_width = 15
gdf2['SUBFRAME_ptp'][(gdf2['CAMERA'] != 'T')] += subframe_width

In [ ]:
#The dissolve step can result in multipolygons, split across 180 or due to labeling errors
multi_idx = (gdf2.geom_type == 'MultiPolygon').nonzero()[0]
multi_idx

In [ ]:
if multi_idx:
    print(gdf2.loc[multi_idx[1]].geom)
    gdf2.loc[multi_idx[1]].geom

In [ ]:
#Need to fix this
#For now, just explode multipolygons to individual polygon records
gdf2 = gdf2.explode()

In [ ]:
gdf.to_file(out_fn, driver='GPKG')

In [ ]:
out_fn=os.path.splitext(out_fn)[0]+'_dissolve.gpkg'
gdf2.to_file(out_fn, driver='GPKG')

In [ ]:
#Now merge and create heatmap
#ogr_merge.sh KH-9_Hexagon_Final_merge.shp *clean_dissolve.gpkg
#gdal_rasterize -burn 1 -tr 0.1 0.1 -ot UInt32 $gdal_opt -a_nodata 0 -add KH-9_Hexagon_Final_merge.shp KH-9_Hexagon_Final_merge_count.tif